In [0]:
#Step 1 - Read the CSV file using the spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:

circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)
])

In [0]:
circuits_df = spark.read.load('dbfs:/FileStore/circuits.csv',format='csv',sep=',',header = True , Schema = circuits_schema  )

In [0]:
circuits_df.show()

+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|circuitId|    circuitRef|                name|    location|  country|     lat|      lng|alt|                 url|
+---------+--------------+--------------------+------------+---------+--------+---------+---+--------------------+
|        1|   albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968| 10|http://en.wikiped...|
|        2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738| 18|http://en.wikiped...|
|        3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|  7|http://en.wikiped...|
|        4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|109|http://en.wikiped...|
|        5|      istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|130|http://en.wikiped...|
|        6|        monaco|   Circuit de Monaco| Monte-Carlo|   Monaco| 43.7347| 

In [0]:
#Step 2 - Select only the required columns

In [0]:
from pyspark.sql.functions import col

In [0]:
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))
     

In [0]:
#Step 3 - Rename the columns as required

In [0]:

circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") 


In [0]:
#Step 4 - Add ingestion date to the dataframe

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
circuits_final_df = circuits_renamed_df.withColumn("ingestion_date" , current_timestamp())

In [0]:
circuits_final_df.show(truncate=False)

+----------+--------------+------------------------------+------------+---------+--------+---------+--------+-----------------------+
|circuit_id|circuit_ref   |name                          |location    |country  |latitude|longitude|altitude|ingestion_date         |
+----------+--------------+------------------------------+------------+---------+--------+---------+--------+-----------------------+
|1         |albert_park   |Albert Park Grand Prix Circuit|Melbourne   |Australia|-37.8497|144.968  |10      |2024-04-22 17:59:07.918|
|2         |sepang        |Sepang International Circuit  |Kuala Lumpur|Malaysia |2.76083 |101.738  |18      |2024-04-22 17:59:07.918|
|3         |bahrain       |Bahrain International Circuit |Sakhir      |Bahrain  |26.0325 |50.5106  |7       |2024-04-22 17:59:07.918|
|4         |catalunya     |Circuit de Barcelona-Catalunya|Montmeló    |Spain    |41.57   |2.26111  |109     |2024-04-22 17:59:07.918|
|5         |istanbul      |Istanbul Park                 |Ista

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/Formula1/processed")

Out[16]: True

In [0]:
circuits_final_df.write.parquet("FileStore/Formula1/processed/circuits") 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-841007799630700>:1
----> 1 circuits_final_df.write.parquet("FileStore/Formula1/processed/circuits")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__ca

In [0]:
%fs ls "FileStore/Formula1/processed"